The goal of this project is to generate a short description based on the content of a news article. Trained on the following CNBC dataset: https://data.world/crawlfeeds/cnbc-news-dataset

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import string
import re
import json
import csv

from transformers import BartForSequenceClassification, Trainer, TrainingArguments, EvalPrediction, AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
from datasets import load_dataset, load_from_disk

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
np.random.seed(0)

In [14]:
""" IMPORTING DATA """

def import_data(file_name):
    # temporarily store data in python lists
    data = []

    with open(file_name, 'r') as csvfile:
        csvreader = csv.reader(csvfile)

        # skip first row because it has labels
        next(csvreader)
        data_indices = [5,9]

        for row in csvreader:
            data.append([row[i] for i in data_indices])

    return data


In [15]:
def clean_text(text):
    pattern = f'[^a-zA-Z0-9\s{re.escape(string.punctuation)}]'
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

In [20]:
data_path = '../../Data/cnbc_news_datase.csv'
processed_data_directory = '../../Data/cnbc_processed/'
data = import_data(data_path)
data = np.array(data)
# column 0 is description, column 1 is content

print(f'raw data: \n{data}')

'''
DATA CLEANING: 

keep only alphanumeric and punctuation
remove entries where data is missing
'''

vectorized_clean = np.vectorize(clean_text)
data = vectorized_clean(data)

rows_with_null = (data == "").any(axis=1)
data = data[~rows_with_null]
labels = np.array(['summary', 'text'])

data = np.vstack((labels, data))
num_test = int(len(data) * 0.1)
random_indices = np.random.choice(data.shape[0], num_test, replace=False)
data_test = data[random_indices]
data_train = np.delete(data, random_indices, axis=0)

train_file = processed_data_directory + 'train.csv'
test_file = processed_data_directory + 'test.csv'

with open(train_file, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerows(data_train)

with open(test_file, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerows(data_test)


raw data: 
[["This is the daily notebook of Mike Santoli, CNBC's senior markets commentator, with ideas about trends, stocks and market statistics."
  "This is the daily notebook of Mike Santoli, CNBC's senior markets commentator, with ideas about trends, stocks and market statistics.A muted, inconclusive bounce that has left the indexes fully within yesterday's low-to-high range all morning so far."]
 ['This commentary originally ran on Facebook. Boris Johnson – The former London mayor and grudging "Leave" supporter-turned enthusiastic "Leave" leader chose the winning side. He\'s now a front-runner to lead the Tory party. \u202a#\u200eHellFreezesOver\u202c Theresa May – Boris Johnson\'s populist approach to the Brexit campaign ticked off quite a few Tory elders, so the real money should be on UK Home Secretary Theresa May to succeed outgoing Prime Minister David Cameron. May is a political heavyweight and is generally respected among the Tories, something Boris Johnson is … um, not.  

The dataset is cleaned and two CSV files are created, splitting the training and testing data.

In [17]:
bart = torch.hub.load('pytorch/fairseq', 'bart.large')
bart.cuda()
bart.train()

def compute_metrics(p: EvalPrediction):
    metric_acc = load_metric("accuracy")
    metric_f1 = load_metric("f1")
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis = 1)
    result = {}
    result["accuracy"] = metric_acc.compute(predictions = preds, references = p.label_ids)["accuracy"]
    result["f1"] = metric_f1.compute(predictions = preds, references = p.label_ids, average = 'macro')["f1"]
    return result

Using cache found in C:\Users\eggyr/.cache\torch\hub\pytorch_fairseq_main


KeyboardInterrupt: 